# Fisrt part

## Exercise 3

``` mysql
CREATE TABLE Flight (
id_flight VARCHAR(20) PRIMARY KEY,
    maximum_capacity SMALLINT CHECK (maximum_capacity >= 0 AND maximum_capacity <= 999),
    current_capacity SMALLINT CHECK (current_capacity >= 0 AND current_capacity <= 999)
    -- Autres colonnes de votre table Flight
);
```


``` mysql
CREATE TABLE Booking(
Id_booking INT AUTO_INCREMENT PRIMARY KEY,
client_name VARCHAR(50),
quantity TINYINT CHECK (quantity>=0 AND quantity<=99),
id_flight VARCHAR(20) NOT NULL,
FOREIGN KEY (id_flight) REFERENCES Flight(id_flight));

```

## Exercise 4

In [4]:
!pip install mariadb

In [2]:
# Module Imports
import mariadb
import time
import random
import threading
from import_sql_dump import import_sql_dump

In [3]:
# Execption used to stop the Notebook cell execution politely
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
# Connect to the server and return a Connection object for the db_name database.
def connectToDB(db_name):
    try:
        return mariadb.connect(
            user="root",
            password="myPa$$",
            host="localhost", # Use of localhost because 1/ we don't have acces to the docker local network 2/ we have made a port redirection from the mariadb server to the host
            port=3306,
            database=db_name
        )
    
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        raise StopExecution
    
conn = connectToDB("Flight_reservation_app")


In [4]:
# A bit of dark magic? Just two intrincated query, one to get the TABLES, one for each TABLE to get the COLUMNS.
# And a clever use of format to have aligned columns.
def showTablesAndColumns(conn):
    cur = conn.cursor()
    # Query the tables
    cur.execute("SHOW TABLES;")
    # For each table
    for table in cur.fetchall():
        print("**",table[0],"**")
        # Defining the output format {:16} indicate at least 16 characters, add spaces if necessary
        outputFormat="{:16}\t{:10}\t{}\t{}\t{}\t{}"
        # Printing the headers
        print(outputFormat.format("Field","Type","Null","Key","Default","Extra"))
    # Query the columns
        cur.execute('SHOW COLUMNS FROM {};'.format(table[0]))
    # For each column
        for attribute in cur.fetchall():
        # Print the attribute (*attribute give the content of the tuple to the format function)
            print(outputFormat.format(*attribute))
showTablesAndColumns(conn)

** Booking **
Field           	Type      	Null	Key	Default	Extra
Id_booking      	int(11)   	NO	PRI	None	auto_increment
client_name     	varchar(50)	YES		None	
quantity        	tinyint(4)	YES		None	
id_flight       	varchar(20)	NO	MUL	None	
** Flight **
Field           	Type      	Null	Key	Default	Extra
id_flight       	varchar(20)	NO	PRI	None	
maximum_capacity	smallint(6)	YES		None	
current_capacity	smallint(6)	YES		None	


In [5]:
def addFlight(id_flight, maximum_capacity):
    conn = connectToDB("Flight_reservation_app")
    cur = conn.cursor()
    try:
        cur.execute(f"INSERT INTO Flight (id_flight, maximum_capacity, current_capacity) VALUES ('{id_flight}', {maximum_capacity}, 0)")
        conn.commit()  # Commit the transaction
    except Exception as e:
        print("Error:", e)  # Handle any exceptions
        conn.rollback()  # Rollback the transaction if an error occurs
    finally:
        conn.close()  # Close the database connection

In [6]:
def cleanDB(id_flight):
    conn = connectToDB("Flight_reservation_app")
    cur = conn.cursor()
    cur.execute("DELETE FROM Booking WHERE id_flight=%s;",(id_flight,))
    cur.execute("UPDATE Flight SET current_capacity=0 WHERE id_flight=%s;",(id_flight,))
    conn.commit() # Without commit the modification are not applied to the database
    conn.close()

In [7]:
def addBooking(id_flight, client_name, quantity):

    #Connection to the database
    conn = connectToDB("Flight_reservation_app")
    cur = conn.cursor()

    #Test if the flight exists
    try:
        # Check if the flight exists
        cur.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE")
        cur.execute("SELECT id_flight, maximum_capacity, current_capacity FROM Flight WHERE id_flight = %s", (id_flight,))
        flight_info = cur.fetchone()

        if flight_info:

            flight_left_seats=flight_info[1]-flight_info[2]
            # Flight exists, proceed with booking
            # Your booking insertion code goes here

            if flight_left_seats>quantity:
                time.sleep(random.randint(3, 5))
                conn.begin()
                #Enough seats available, proceed with booking
                cur.execute("INSERT INTO Booking (id_flight, client_name, quantity) VALUES (%s, %s, %s)", (id_flight, client_name, quantity))
                #return (True, "Booking added successfully")
                cur.execute("UPDATE Flight SET current_capacity = current_capacity + %s WHERE id_flight= %s", (quantity, id_flight))
                conn.commit()  # Commit the transaction
                return  (False,"Reserved")
            
            else:
                # Not enough seats available
                return (False, "Full")
            
        else:
            # Flight doesn't exist
            return (False, "Invalid")
        
    except Exception as e:
        print("Error:", e)  # Handle any exceptions
        conn.rollback()  # Rollback the transaction if an error occurs
        return  (True,"Error")

    finally:
        conn.close()  # Close the database connection

In [43]:
#Test for the creation of new flight

addFlight("LH6795",10)
print(addBooking("LH6795","Client 0",1))
print(addBooking("LH6795","Client 1",4))
print(addBooking("LH6795","Client 2",3))
print(addBooking("LH6795","Client 3",3))

Error: Duplicate entry 'LH6795' for key 'PRIMARY'
(False, 'Reserved')
(False, 'Reserved')
(False, 'Reserved')
(False, 'Full')


## Exercise 6

In [64]:
def processBooking(id_flight, client_name, quantity):
    print("Processing Flight",id_flight,"for",client_name,":", quantity,"seats.")
    retry,status = addBooking(id_flight,client_name,quantity)

    while retry :
        print("Retry Flight",id_flight,"for",client_name,":",quantity,"seats.")
        retry,status = addBooking(id_flight,client_name,quantity)
        print(client_name,":",status)

    print(client_name,":",status, "Should retry:",retry)

In [65]:
# Test parameters
id_flight="LH6795"
base_name="Client "
# Clean the db for the test
cleanDB(id_flight)
# Creating several Thread representing several client that try to book simultaneously a random number of seat
jobs=[]
for i in range(5):
    jobs.append(threading.Thread(target=processBooking,args=(id_flight,base_name+str(i), random.randint(1,4))))
# Start each job, giving 0.3s of delay between each
for job in jobs:
    job.start()
    time.sleep(0.3)

# Wait for all the thread to finish
for job in jobs:
    job.join()
print("Finished")

Processing Flight LH6795 for Client 0 : 2 seats.
Processing Flight LH6795 for Client 1 : 1 seats.
Processing Flight LH6795 for Client 2 : 1 seats.
Processing Flight LH6795 for Client 3 : 3 seats.
Processing Flight LH6795 for Client 4 : 3 seats.
Error: Lock wait timeout exceeded; try restarting transaction
Retry Flight LH6795 for Client 0 : 2 seats.
Error: Lock wait timeout exceeded; try restarting transaction
Error: Lock wait timeout exceeded; try restarting transaction
Error: Lock wait timeout exceeded; try restarting transaction
Retry Flight LH6795 for Client 2 : 1 seats.
Retry Flight LH6795 for Client 3 : 3 seats.
Retry Flight LH6795 for Client 1 : 1 seats.
Client 4 : Reserved Should retry: False


### Exercise 6.1

I changed the time.sleep to 1 because i had some trouble.

All client made their reservation, but there is more booked seat than avaible. In the sequential tes of exercise 5 for each new booking, a test is made to make sure that there are available seats, looks like it is note the case here because all reservatio are made at the same time. 

### Exercise 6.2


When there is not a time sleep or when it is to small, not all client were able to make their reservation. But there isn't any real logic because many could have made a booking.

the error message is : *Error: Lock wait timeout exceeded; try restarting transaction*


The error message "Lock wait timeout exceeded; try restarting transaction" indicates that one or more transactions in the system are waiting for a lock on a resource, but the wait time has exceeded the timeout limit set by the database system. This could be due to contention for locks caused by multiple transactions trying to access the same resource concurrently.

### Exercise 6.3

In [53]:
conn = connectToDB("Flight_reservation_app")
cur = conn.cursor()

cur.execute("SELECT @@tx_ISOLATION;")
print (cur.fetchone()[0])

conn.close()

REPEATABLE-READ


In a REPEATABLE READ isolation level:

When a transaction starts, it sees a consistent snapshot of the database as it was at the start of the transaction.
Any data read by the transaction will remain consistent throughout the transaction, even if other transactions modify the data.
However, new data inserted by other transactions after the start of the current transaction will not be visible to the current transaction until it completes and starts a new transaction.

### Exercise 6.4

### Exercise 6.5 

### Exercise 6.6

In [ ]:
conn = connectToDB("flight_reservation")
cur = conn.cursor()
cur.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE")
cur.execute("SELECT COUNT(*) FROM Flight WHERE id_flight=?",(id_flight,))

In summary, transactions ensure data integrity in databases, while isolation levels control how transactions interact. Choices like READ UNCOMMITTED, READ COMMITTED, REPEATABLE READ, and SERIALIZABLE offer different trade-offs between consistency and concurrency.

SERIALIZABLE provides the strongest consistency but may limit concurrency. REPEATABLE READ balances between consistency and concurrency. Lower levels like READ COMMITTED and READ UNCOMMITTED prioritize concurrency over consistency.

Choosing the right isolation level depends on the application's requirements and performance considerations. While SERIALIZABLE offers strong consistency, it may not always be necessary or practical in high-concurrency scenarios. It's crucial to weigh the trade-offs and select the appropriate level for each situation.

In [48]:
conn.close()

# Seconde Part

## Exercise 7.1

![alt text](<ex_7_1_error image.png>)

When you encounter the error message "Warning: POST Content-Length of [X] bytes exceeds the limit of [Y] bytes" during the import process in PhpMyAdmin, it indicates that the size of the SQL dump file being uploaded exceeds the maximum allowed size for POST requests in the PHP configuration.

To fix this issue and successfully import the SQL dump file into your database, you can adjust the following PHP configuration settings:

post_max_size: This setting determines the maximum size of POST data that PHP will accept. Increase this value to accommodate the size of your SQL dump file.
upload_max_filesize: This setting controls the maximum size of uploaded files. Make sure it is set to a value equal to or greater than the size of your SQL dump file.
To adjust these settings, you'll need access to the PHP configuration file (php.ini) on your web server. Here's how you can do it:

Locate the php.ini file on your server. The location may vary depending on your server configuration, but common paths include /etc/php.ini or /etc/php/[version]/php.ini.
Open the php.ini file in a text editor.
Search for the post_max_size and upload_max_filesize directives.
Increase the values of these directives to accommodate the size of your SQL dump file. For example:
makefile
Copy code
post_max_size = 20M
upload_max_filesize = 20M
Save the changes to the php.ini file.
Restart your web server to apply the changes.

an other solution solution is to compress it to make smaller.

## Exercise 7.2

The SQL request I choose to get familiar with the values and the link between tables:

```mysql
SHOW TABLES;
```

This query lists all the tables available in the benchmarking database. It provides an overview of the database schema and the different entities stored within it. By examining the table names, you can identify the entities present in the database and understand the scope of data stored.

```mysql
DESCRIBE table_name;
```

This query displays the structure of the Booking table, including the names, data types, and constraints of its columns. By examining the table structure, you can understand the attributes and properties associated with bookings in the database. This information is crucial for querying and manipulating booking data effectively.

```mysql
SELECT * FROM table_name LIMIT 10;
```

This query retrieves a sample of data from the specified table, limiting the result to the first 10 rows. Examining sample data provides insights into the actual values stored in the chosen table and helps understand its relationships with other tables in the database.

## Exercise 8.1

Check [import_sql_dump.py](import_sql_dump.py).

## Exercise 8.2

In [8]:
from import_sql_dump import import_sql_dump
dump_file_template = 'DUMP.sql.zip'

num_cycles = 5

conn = connectToDB('benchmarking')

# Perform import/export cycles
for i in range(num_cycles):
    import_sql_dump(dump_file_template,conn,f'dump_{i+1}.sql')


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 95: character maps to <undefined>

## Exercise 9.1

In [9]:
cur=conn.cursor()

In [10]:
cur.execute("SELECT Reserve.seat, Booking.flight_id FROM Reserve JOIN Booking ON Reserve.booking_id = Booking.booking_id GROUP BY Reserve.seat, Booking.flight_id HAVING COUNT(*) >1;")
print(f'Here are all the seat per flight ID that have more several passenger on assigned to it.')
for i in cur.fetchall():
    print (f'In flight {i[1]}, seat {i[0]}')

Here are all the seat per flight ID that have more several passenger on assigned to it.
In flight 92, seat 10A
In flight 124, seat 10A
In flight 143, seat 10A
In flight 154, seat 10A
In flight 253, seat 10A
In flight 7005, seat 10A
In flight 7822, seat 10A
In flight 7896, seat 10A
In flight 9600, seat 10A
In flight 13977, seat 10A
In flight 14428, seat 10A
In flight 14639, seat 10A
In flight 16159, seat 10A
In flight 18152, seat 10A
In flight 20056, seat 10A
In flight 100, seat 10B
In flight 119, seat 10B
In flight 152, seat 10B
In flight 193, seat 10B
In flight 236, seat 10B
In flight 238, seat 10B
In flight 240, seat 10B
In flight 5950, seat 10B
In flight 7933, seat 10B
In flight 8654, seat 10B
In flight 8709, seat 10B
In flight 8961, seat 10B
In flight 13197, seat 10B
In flight 15131, seat 10B
In flight 18346, seat 10B
In flight 19564, seat 10B
In flight 19622, seat 10B
In flight 19662, seat 10B
In flight 20070, seat 10B
In flight 20114, seat 10B
In flight 20682, seat 10B
In flight 

## Exercise 9.2

This is the QUERY to have all information diplayed:

```mysql
SELECT 
    r1.booking_id AS booking_id_1,
    r1.passenger_id,
    f1.flight_id AS flight_id_1,
    f1.flight_day AS flight_day_1,
    f1.flight_hour AS flight_hour_1,
    fi1.flight_duration AS duration_1,
    f1.route AS route_1,
    f2.flight_id AS flight_id_2,
    f2.flight_day AS flight_day_2,
    f2.flight_hour AS flight_hour_2,
    fi2.flight_duration AS duration_2,
    f2.route AS route_2
FROM 
    Reserve r1
INNER JOIN 
    Reserve r2 ON r1.passenger_id = r2.passenger_id AND r1.booking_id != r2.booking_id
INNER JOIN 
    Booking b1 ON r1.booking_id = b1.booking_id
INNER JOIN 
    Booking b2 ON r2.booking_id = b2.booking_id
INNER JOIN 
    Flight f1 ON b1.flight_id = f1.flight_id
INNER JOIN 
    Flight f2 ON b2.flight_id = f2.flight_id
INNER JOIN
    Flight_information fi1 ON f1.route = fi1.route
INNER JOIN
    Flight_information fi2 ON f2.route = fi2.route
WHERE 
    f1.flight_day = f2.flight_day
    AND (
        (f1.flight_hour <= f2.flight_hour AND f1.flight_hour + fi1.flight_duration > f2.flight_hour) 
        OR 
        (f2.flight_hour <= f1.flight_hour AND f2.flight_hour + fi2.flight_duration > f1.flight_hour)
    );
```

In [16]:

cur.execute("SELECT r1.passenger_id, f1.flight_id AS flight_id_1, f1.flight_day AS flight_day_1, f1.flight_hour AS flight_hour_1, fi1.flight_duration AS duration_1, f1.route AS route_1, f2.flight_id AS flight_id_2, f2.flight_day AS flight_day_2, f2.flight_hour AS flight_hour_2, fi2.flight_duration AS duration_2, f2.route AS route_2 FROM Reserve r1 INNER JOIN Reserve r2 ON r1.passenger_id = r2.passenger_id AND r1.booking_id != r2.booking_id INNER JOIN Booking b1 ON r1.booking_id = b1.booking_id INNER JOIN Booking b2 ON r2.booking_id = b2.booking_id INNER JOIN Flight f1 ON b1.flight_id = f1.flight_id INNER JOIN Flight f2 ON b2.flight_id = f2.flight_id INNER JOIN Flight_information fi1 ON f1.route = fi1.route INNER JOIN Flight_information fi2 ON f2.route = fi2.route WHERE f1.flight_day = f2.flight_day AND ( (f1.flight_hour <= f2.flight_hour AND f1.flight_hour + fi1.flight_duration > f2.flight_hour) OR (f2.flight_hour <= f1.flight_hour AND f2.flight_hour + fi2.flight_duration > f1.flight_hour));")

#print(cur.fetchall())
print(f'Here are all the passenger ID that made  booked 2 different flights overlapping each other in time.')
for i in cur.fetchall():
    print (f'Passenger n°{i[0]} booked a flight n°{i[1]} planned {i[2]} at {i[3]} making the route {i[5]} in {i[4]} hours but also the flight n°{i[6]} planned {i[7]} at {i[8]} making the route {i[10]} in {i[9]} hours')

Here are all the passenger ID that made  booked 2 different flights overlapping each other in time.
Passenger n°22 booked a flight n°4 planned Mon at 19 making the route AKLDEL in 5.52 hours but also the flight n°8543 planned Mon at 17 making the route DMKPVG in 5.33 hours
Passenger n°22 booked a flight n°4 planned Mon at 19 making the route AKLDEL in 5.52 hours but also the flight n°2624 planned Mon at 16 making the route CGKHND in 7.57 hours
Passenger n°9 booked a flight n°7 planned Wed at 15 making the route AKLDEL in 5.52 hours but also the flight n°22844 planned Wed at 13 making the route PENTPE in 4.67 hours
Passenger n°2 booked a flight n°16 planned Sat at 7 making the route AKLDEL in 5.52 hours but also the flight n°23172 planned Sat at 2 making the route PERPVG in 5.62 hours
Passenger n°36 booked a flight n°28 planned Mon at 8 making the route AKLICN in 6.62 hours but also the flight n°12453 planned Mon at 14 making the route HNDPER in 7.57 hours
Passenger n°65 booked a flight

## Exercise 9.3

Ensure that each passenger's email address in the Passenger table is unique.

<small>It is important because it is the only way to communicate with them and them to have an access to the web service.</small>

In [18]:
cur.execute("SELECT email, COUNT(*) FROM Passenger GROUP BY email HAVING COUNT(*) >1;")

print(cur.fetchall())
print(f"All email are unique.")


[]
All email are unique.


Ensure that all necessary data is present and complete for all passenger.

<small>A missing data can be very problematic espacially the lastname and firstname has all seats are nominated.</small>



In [20]:

cur.execute("SELECT * FROM Passenger WHERE lastname IS NULL OR firstname IS NULL OR email IS NULL;")

print(cur.fetchall())
print(f"All information are filled in.")


[]
All information are filled in.


## Exercise 10

Same seat query: 0.1856 seconds<br>
Overlapping flights: 0.0352 seconds<br>
Unique email: 0.1297 seconds<br>
Missing value: 0.0003 seconds<br>